In [1]:
import cv2
image_path = './testdata/ex.png'

img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
guidance = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(img.shape, guidance.shape)

(316, 316, 3) (316, 316)


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img_rgb)
plt.axis("off")  # 移除座標軸
plt.show()

In [52]:
sigma_r = 0.1
sigma_s = 3
wndw_size = 6*sigma_s+1
pad_w = 3*sigma_s

BORDER_TYPE = cv2.BORDER_REFLECT
padded_img = cv2.copyMakeBorder(img_rgb, pad_w, pad_w, pad_w, pad_w, BORDER_TYPE)
padded_guidance = cv2.copyMakeBorder(guidance, pad_w, pad_w, pad_w, pad_w, BORDER_TYPE)
print(padded_img.shape, padded_guidance.shape)

(334, 334, 3) (334, 334)


In [4]:
import numpy as np

padded_guidance = padded_guidance.astype('float64') / 255
padded_img = padded_img.astype('float64')
output = np.zeros(img_rgb.shape)

In [ ]:
GaussianSpatial = np.zeros((wndw_size, wndw_size))
for i in range(wndw_size):
    for j in range(wndw_size):
        GaussianSpatial[i, j] = np.exp(np.divide(np.square(i - pad_w) + np.square(j - pad_w), -2 * np.square(sigma_s)))

# x, y = np.meshgrid(np.arange(self.wndw_size) - self.pad_w, np.arange(self.wndw_size) - self.pad_w, indexing='ij')
# GaussianSpatial = np.exp(-(x**2 + y**2) / (2 * self.sigma_s**2))

In [44]:
for i in range(pad_w, padded_guidance.shape[0] - pad_w):
    for j in range(pad_w, padded_guidance.shape[1] - pad_w):
        Tp = padded_guidance[i, j]
        Tq = padded_guidance[i - pad_w: i + pad_w + 1, j - pad_w: j + pad_w + 1]
        power = np.divide(np.square(Tq - Tp), -2 * np.square(sigma_r))
        if len(power.shape) == 3:
            power = power.sum(axis=2)
        GaussianRange = np.exp(power)

        G = np.multiply(GaussianSpatial, GaussianRange)
        W = G.sum()

        Iq = padded_img[i - pad_w: i + pad_w + 1, j - pad_w: j + pad_w + 1] # (19, 19, 3)
        for c in range(img_rgb.shape[2]):
            output[i - pad_w, j - pad_w, c] = np.multiply(G, Iq[:,:,c]).sum() / W

output = np.clip(output, 0, 255).astype(np.uint8)

### Main.py section

In [25]:
""" 讀取設定檔中的灰階轉換參數和 sigma 值 """
with open('./testdata/1_setting.txt', 'r') as f:
    lines = f.readlines()

grayscale_params = []
sigma_s, sigma_r = None, None

# for line in lines:
#     values = line.strip().split(',')
#     if "sigma_s" in line:
#         sigma_s = float(values[1])  # 解析 sigma_s
#         sigma_r = float(values[3])  # 解析 sigma_r
#     else:
#         grayscale_params.append(list(map(float, values)))  # 解析 R, G, B 權重

# return np.array(grayscale_params), sigma_s, sigma_r

In [26]:
import numpy as np

for line in lines:
    values = line.strip().split(',')
    if values[0] == 'R':
        continue
    elif "sigma_s" in line:
        sigma_s = float(values[1])
        sigma_r = float(values[3])
    else:
        grayscale_params.append(list(map(float, values)))  # 解析 R, G, B 權重
grayscale_params = np.array(grayscale_params)

In [51]:
import cv2

""" 根據設定的轉換參數生成灰階影像 """
img = cv2.imread('./testdata/1.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray_images = []
for params in grayscale_params:
    gray_img = np.dot(img_rgb, params).astype(np.uint8)  # RGB 加權求和轉換
    gray_images.append(gray_img)

print(gray_images[0].shape, img_gray.shape)

(300, 400) (300, 400)


In [50]:
for i in range(pad_w, padded_guidance.shape[0] - pad_w):
    for j in range(pad_w, padded_guidance.shape[1] - pad_w):
        Tp = padded_guidance[i, j]
        Tq = padded_guidance[i - pad_w: i + pad_w + 1, j - pad_w: j + pad_w + 1]
        power = np.divide(np.square(Tq - Tp), -2 * np.square(sigma_r))
        if len(power.shape) == 3:
            power = power.sum(axis=2)
        GaussianRange = np.exp(power)

        G = np.multiply(GaussianSpatial, GaussianRange)
        W = G.sum()

        Iq = padded_img[i - pad_w: i + pad_w + 1, j - pad_w: j + pad_w + 1] # (19, 19, 3)
        for c in range(img_rgb.shape[2]):
            output[i - pad_w, j - pad_w, c] = np.multiply(G, Iq[:,:,c]).sum() / W

output1 = np.clip(output, 0, 255).astype(np.uint8)

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view
H, W = padded_guidance.shape[:2]
img_H, img_W = H - 2 * pad_w, W - 2 * pad_w

Tq = sliding_window_view(padded_guidance, (2 * pad_w + 1, 2 * pad_w + 1))  # (H-2pad, W-2pad, wndw_size, wndw_size)
Iq = sliding_window_view(padded_img, (2 * pad_w + 1, 2 * pad_w + 1, 3))    # (H-2pad, W-2pad, wndw_size, wndw_size, 3)

# 生成座標網格 (i, j)
i, j = np.meshgrid(np.arange(img_H), np.arange(img_W), indexing='ij')

# 取得中心像素 Tp (i, j 對應的 guidance map)
Tp = padded_guidance[pad_w:H-pad_w, pad_w:W-pad_w]  # (img_H, img_W)

# 計算 Gaussian Range (取代 for 迴圈)
power = np.divide(np.square(Tq - Tp[:, :, None, None]), -2 * np.square(sigma_r))  # (img_H, img_W, wndw_size, wndw_size)

# 若是多通道影像 (RGB)
if len(power.shape) == 3:
    power = power.sum(axis=2)  # 對 RGB 通道進行累加

GaussianRange = np.exp(power)  # (img_H, img_W, wndw_size, wndw_size)

G = GaussianSpatial[None, None, :, :] * GaussianRange  # (img_H, img_W, wndw_size, wndw_size)
W = G.sum(axis=(2, 3), keepdims=True)  # 加總權重 (img_H, img_W, 1, 1)

# 計算最終濾波結果（對 R, G, B 分開計算）
output2 = np.sum(G[..., None] * Iq, axis=(2, 3)) / W  # (img_H, img_W, 3)

